# Getting Started
## Starting point : SparkSession

- Spark에서의 모든 functionality에 들어가는 entry point는 SparkSession이라는 클래스이다.
- 그래서 기본 SparkSession을 만들기 위해 ``SparkSession.builder``사용
- 만약 가상환경에서 Pyspark를 설치했다면 실행도 가상환경에서 실행해주어야 함!
    * **왜냐하면 Pyspark 설치했을 때의 Python 버전과 실행했을 때의 Python 버전이 일치해야 함! 아니면 에러 발생!**

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .config('spark.some.config.option', 'some-value')\
        .getOrCreate()

In [2]:
### Create json file using spark
# sparkContext로 객체 생성
sc = spark.sparkContext

# json 파일 읽어들이기
path = '/Users/younghun/Desktop/gitrepo/TIL/pyspark/people.json'
peopleDF = spark.read.json(path)

# printSchema()로 json파일의 스키마 형태 볼수 있음
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [3]:
# 데이터프레임을 사용하는 임시의 view(가상의 테이블) 생성
peopleDF.createOrReplaceTempView("people")

# spark에서 제공하는 sql 메소드를 이용해 쿼리 날리기
# 쿼리문에서 people 테이블은 위에서 만들었던 view 테이블임!
teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
teenagerNamesDF.show()

+------+
|  name|
+------+
|Justin|
+------+



In [4]:
# 또한 데이터프레임은 RDD[String] 자료구조를 이용해서 json 데이터셋을 데이터프레임으로 만들 수 있음
jsonStrings = ['{"name": "Yin", "address":{"city":"Columbus", "state":"Ohio"}}']
# json -> RDD형식으로 만들기
otherPeopleRDD = sc.parallelize(jsonStrings)
# json파일 읽어오기
otherPeople = spark.read.json(otherPeopleRDD)
otherPeople.show()

+----------------+----+
|         address|name|
+----------------+----+
|[Columbus, Ohio]| Yin|
+----------------+----+



## Creating DataFrames

In [5]:
df = spark.read.json(path)
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Untyped Dataset Operations
- a.k.a DataFrame Operations
<br><br>
- 데이터프레임의 칼럼에 접근하기 위한 방법
    * df.column 
    * df['column'] : 이 방법이 사람들이 더 많이 사용
    

In [6]:
# 데이터프레임의 스키마 미리보기
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### select

In [7]:
# name 칼럼 select 해서 살펴보기
df.select('name').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [8]:
# name과 age 칼럼값들을 보는데, age에 1을 더하는 연산 취해 새로운 칼럼 만들어 select
# 단, null값에는 더해도 null
df.select(df['name'], df['age']+1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



### filter

- 특정 조건을 만족하는 행 추출하기

In [9]:
df.filter(df['age'] > 20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



### groupBy
- 특정 칼럼으로 그룹핑해서 count하기

In [10]:
df.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [12]:
df.groupBy('name').count().show()

+-------+-----+
|   name|count|
+-------+-----+
|Michael|    1|
|   Andy|    1|
| Justin|    1|
+-------+-----+



## Running SQL Queries

- spark로 sql 쿼리를 날리면 데이터프레임 형태로 반환함

In [13]:
# view로 가상의 테이블 생성
df.createOrReplaceTempView('people') # people = 테이블 이름

sqlDF = spark.sql('SELECT * FROM people')
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Global Temporary View

- 일반 Temporary view의 가상테이블은 ``SparkSession``이 종료되면 삭제됨
- 하지만 모든 ``SparkSession``들 간에 view 가상 테이블을 공유하고 싶다면 Global Temporary view 사용
- Global Temporary view는 시스템 내부에 보존되는 데이터베이스인 ``global_temp``와 연결되어 있어서 테이블 앞에 ``gobal_temp``를 붙여주어야 한다<br>ex) SELECT * FROM global_temp.view1

In [14]:
# Global Temporary View 생성
df.createOrReplaceGlobalTempView('people')

sqlDF = spark.sql('SELECT * FROM global_temp.people')
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Reflection을 사용해 Schema 만들기

- Row 메소드 사용

In [33]:
from pyspark.sql import Row
# SparkContext 객체 만들기
sc = spark.sparkContext

# txt file 한 줄씩 읽어오기
lines = sc.textFile('./people.txt')
print('lines의 type:', type(lines)) # RDD 자료구조

# 읽어온 한 줄마다 map함수를 사용해 전처리 - ,구분자로 구분해 리스트로 담기
parts = lines.map(lambda l: l.split(','))

# Row를 통해 name이라는 칼럼명에는 P리스트의 첫 번째 값을 넣기
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# 스키마를 형성하고 데이터프레임으로 생성
schemaPeople = spark.createDataFrame(people)
# View 테이블로 만들기
schemaPeople.createOrReplaceTempView('people')

# 만든 View 테이블에서 원하는 데이터 추출 -> SQL결과는 데이터프레임 객체로 반환
teenagers = spark.sql('SELECT name FROM people WHERE age >= 13 AND age <= 19')
print('teenagers type:', type(teenagers))

# RDD는 데이터프레임의 값(content)를 반환함
# 여기서 p는 teenagers라는 데이터프레임의 각 row를 의미
teenNames = teenagers.rdd.map(lambda p: 'Name: ' + p.name).collect()
print('teenNames type:', type(teenNames)) # list로 반환됨!
print(teenNames)

for name in teenNames:
    print(name)

lines의 type: <class 'pyspark.rdd.RDD'>
teenagers type: <class 'pyspark.sql.dataframe.DataFrame'>
teenNames type: <class 'list'>
['Name: Justin']
Name: Justin


## schema를 programmatical하게 명시하는 방법

1. Tuple, List의 RDD를 생성
2. 1단계에서 만들어진 RDD안에 존재하는 tuple, list의 구조를 매칭시키는 ``StructType``에 의해 나타내지는 스키마를 생성
3. SparkSession이 제공하는 메소드인 ``createDataFrame``을 활용해 2단계에서 만든 RDD에 스키마를 적용

In [34]:
from pyspark.sql.types import *

# SparkContext 객체 생성
sc = spark.sparkContext

# txt file 읽어오기
lines = sc.textFile('./people.txt')
parts = lines.map(lambda l: l.split(','))

## Step 1 ## => value들 처리
# 각 라인을 tuple( , ) 형태로 convert 해주기 
people = parts.map(lambda p: (p[0], p[1].strip())) # name에서 공백 strip

## Step 2 ## => Schema들 처리
# 문자열로 인코딩된 스키마
schemaString = "name age"
# schemaString 요소를 loop돌면서 StructField로 만들기
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
# StructField 여러개가 있는 리스트를 StrucType으로 만들기!
schema = StructType(fields)

## Step 3 ## => value와 schema 활용해 DataFrame 생성
# 위에서 만든 schema를 RDD의 schema로 적용
schemaPeople = spark.createDataFrame(people, schema)

# View Table 생성해 쿼리 날려서 데이터 추출해보기
schemaPeople.createOrReplaceTempView('people')
results = spark.sql("SELECT name FROM people")
results.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



- ``count(), countDistince(), avg(), max(), min() 등``과 같은 집계 함수도 존재
- 사용자가 직접 집계함수 customizing 할 수도 있음

In [36]:
# 연습
sc = spark.sparkContext

path = '/Users/younghun/Desktop/gitrepo/TIL/pyspark/people.json'
peopleDF = spark.read.json(path)
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [37]:
peopleDF.createOrReplaceTempView('people')
res = spark.sql('SELECT MAX(age) FROM people')
res.show()

+--------+
|max(age)|
+--------+
|      30|
+--------+

